In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms
import timm
from PIL import Image
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, jaccard_score, matthews_corrcoef, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import label_binarize
import tqdm
from tqdm import tqdm
import joblib
import shap

In [2]:
matplotlib.use('Agg')

In [3]:
# Device Setup

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
# Constants 

MEAN = 0.2956
STD = 0.3069

transform = transforms.Compose([
    transforms.Resize((224, 224), interpolation=Image.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(mean=[MEAN] * 3, std=[STD] * 3)
])

data_dir = '/kaggle/input/fdata-adni-dataset/AugmentedAlzheimerDataset'

SWIN_PATH = '/kaggle/input/swin-e-adni/pytorch/default/1/swin_model.pth'
VGG_PATH = '/kaggle/input/vgg-adni/pytorch/default/1/vgg_model.pth'

RANDOM_SEED = 43
TEST_SIZE = 0.15
VAL_SIZE = 0.15

NUM_CLASSES = 4
CLASSES = ['AD', 'CN', 'EMCI', 'LMCI']

In [5]:
# Seeds
torch.manual_seed(RANDOM_SEED)
generator = torch.Generator().manual_seed(RANDOM_SEED) # Surity

In [6]:
# Datasets

master = datasets.ImageFolder(root=data_dir, transform=transform)

total_size = len(master)
test_size = int(TEST_SIZE * total_size)
val_size = int(VAL_SIZE * total_size)
train_size = total_size - test_size - val_size

train_set, val_set, test_set = random_split(master, [train_size, val_size, test_size], generator=generator)

In [7]:
print(f'Train: {len(train_set)}\nVal: {len(val_set)}\nTest: {len(test_set)}')

Train: 23790
Val: 5097
Test: 5097


In [8]:
# Models

swin = torch.load(SWIN_PATH)
vgg = torch.load(VGG_PATH)

swin.to(device)
vgg.to(device)

swin.eval()
vgg.eval()

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possibl

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [9]:
for module in vgg.modules():
    if isinstance(module, torch.nn.ReLU):
        module.inplace = False

In [10]:
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)
batch = next(iter(test_loader))
images, _ = batch
images = images.view(64, 3, 224, 224).to(device)

background = images[:32]
test_images = images[32:34]

explainer = shap.GradientExplainer(vgg, images.clone())
shap_values = explainer.shap_values(test_images.clone().detach())


In [11]:
shap_numpy = [np.swapaxes(np.swapaxes(s, 1, -1), 1, 2) for s in shap_values]
test_numpy = np.swapaxes(np.swapaxes(test_images.cpu().numpy(), 1, -1), 1, 2)

shap.image_plot(shap_numpy, -test_numpy, show=False)

fig = plt.gcf()
axes = fig.get_axes()

for i in range(NUM_CLASSES):
    ax = axes[1 + i]
    pos = ax.get_position()
    fig.text(
        x=pos.x0 + pos.width/2,
        y=pos.y1 + 0.05,
        s=CLASSES[i],
        ha="center",
        va="bottom",
        fontsize=14,
        fontweight="bold"
    )

In [12]:
plt.savefig('SHAP_VGG.pdf', format='pdf', dpi=1200, bbox_inches='tight')
plt.close()

In [13]:
meta_model = joblib.load('/kaggle/input/mlr-adni/scikitlearn/default/1/mlr_meta_adni (1).pkl')

In [14]:
swin_pred_prob_list_test = []
vgg_pred_prob_list_test = []

swin_outputs = swin(images)
swin_probs = torch.softmax(swin_outputs, dim=1)
swin_pred_prob_list_test.append(swin_probs.detach().cpu().numpy())
        
vgg_outputs = vgg(images)
vgg_probs = torch.softmax(vgg_outputs, dim=1)
vgg_pred_prob_list_test.append(vgg_probs.detach().cpu().numpy())

swin_pred_prob_test = np.concatenate(swin_pred_prob_list_test, axis=0)
vgg_pred_prob_test = np.concatenate(vgg_pred_prob_list_test, axis=0)

combined_features_test = np.concatenate((swin_pred_prob_test, vgg_pred_prob_test), axis=1)

In [15]:
feature_names = ['Swin AD', 'Swin CN', 'Swin EMCI', 'Swin LMCI', 'VGG AD', 'VGG CN', 'VGG EMCI', 'VGG LMCI']

In [16]:
pred_class = CLASSES[meta_model.predict(combined_features_test[-1:])[0]]
pred_class

'EMCI'

In [17]:
from lime.lime_tabular import LimeTabularExplainer

explainer = LimeTabularExplainer(
    training_data=combined_features_test,
    feature_names=feature_names,
    class_names=CLASSES,
    mode='classification',
    discretize_continuous=False
)

explanation = explainer.explain_instance(combined_features_test[-1], meta_model.predict_proba, top_labels=1)

top_class = explanation.available_labels()[0]

weights = dict(explanation.as_list(label=top_class))

plt.figure(figsize=(8, 5))
plt.barh(list(weights.keys()), list(weights.values()), color=['blue' if v > 0 else 'red' for v in weights.values()])
plt.xlabel('Feature Contribution')
plt.ylabel('Feature')
plt.title(f'LIME Explaination for {pred_class} Prediction')
plt.axvline(0, color='black', linestyle='dashed', linewidth=1)
plt.savefig('LIME_MLR.pdf', format='pdf', bbox_inches='tight')